In [13]:
import numpy as np # linear algebra
import os
from PIL import Image
#from skimage.transform import resize
from random import shuffle

In [14]:
# define paths
train_path = "../train/"
test_path = "../test/"
add_data_path = "../flickr_images/"

# print mobile models
models = os.listdir(train_path)
print(models)


['Sony-NEX-7', 'HTC-1-M7', 'Motorola-Droid-Maxx', 'iPhone-6', 'Motorola-Nexus-6', 'Samsung-Galaxy-S4', 'Motorola-X', 'LG-Nexus-5x', 'Samsung-Galaxy-Note3', 'iPhone-4s']


In [15]:
# Cropping function
def center_crop(img_path, new_width, new_height):
        im = Image.open(img_path)
        width, height = im.size   # Get dimensions
        left = (width - new_width)/2
        top = (height - new_height)/2
        right = (width + new_width)/2
        bottom = (height + new_height)/2
        result = im.crop((left, top, right, bottom))
        return result        

In [16]:
# transform images to 512x512 (cropping from the center)
new_width = 512
new_height = 512

for folder in os.listdir(train_path):
    print(folder)
    for pic in os.listdir(train_path + folder + "/"):
        img_path = train_path + folder + "/" + pic
        img_cropped = center_crop(img_path, 512, 512)
        directory = "../train_cropped/" + folder
        # create dir if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)
        img_cropped.save("../train_cropped/" + folder + "/cropped" + "_" + pic )

Sony-NEX-7
HTC-1-M7
Motorola-Droid-Maxx
iPhone-6
Motorola-Nexus-6
Samsung-Galaxy-S4
Motorola-X
LG-Nexus-5x
Samsung-Galaxy-Note3
iPhone-4s


In [17]:
import shutil
import numpy as np

def create_if_not_exist(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def create_train_val_test_dir(source, train_perc, val_perc):
    # create train, validation and test set
    source = "../train_cropped"
    train_set = "../train_set"
    val_set = "../val_set"
    test_set = "../test_set"

    # create dir if it doesn't exist
    for directory in [train_set, val_set, test_set]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    folders = os.listdir(source)

    for f in folders:
        val_dir = val_set + '/'+ f + "/"
        test_dir = test_set + '/'+ f + "/"
        train_dir = train_set + '/'+ f + "/"

        create_if_not_exist(val_dir)
        create_if_not_exist(test_dir)
        create_if_not_exist(train_dir)

        for pic in os.listdir(source + "/" + f + "/"):
            rand_num = np.random.rand(1)
            if rand_num <= train_perc:
                shutil.copy(source + '/'+ f + "/" + pic, val_dir)
            elif train_perc< rand_num <= train_perc + val_perc:
                shutil.copy(source + '/'+ f + "/" + pic, test_dir)
            else:
                shutil.copy(source + '/'+ f + "/" + pic, train_dir)
            
create_train_val_test_dir(train_cropped, 0.2, 0.1)
        

In [59]:
# First Experiment
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(512, 512, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# # # the model so far outputs 3D feature maps (height, width, features)

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [62]:
# Image Augmentation

# resizing (via bicubic interpolation) by a factor of 0.5
# resizing (via bicubic interpolation) by a factor of 0.8
# resizing (via bicubic interpolation) by a factor of 1.5
# resizing (via bicubic interpolation) by a factor of 2.0
# gamma correction using gamma = 0.8
# gamma correction using gamma = 1.2



from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip = True,
        fill_mode='nearest')

In [ ]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip = True,
        fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator( 
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip = True,
        fill_mode='nearest')

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_set,  # this is the target directory
        target_size=(512, 512),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        val_set,
        target_size=(512, 512),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_set,
        target_size=(512, 512),
        batch_size=batch_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch=5, # // batch_size,
        epochs=25,
        validation_data=validation_generator,
        validation_steps=20) #00 // batch_size)

model.save_weights('first_try.h5')  # always save your weights after training or during training

Found 1924 images belonging to 10 classes.
Found 557 images belonging to 10 classes.
Found 269 images belonging to 10 classes.
Epoch 1/25
4/5 [=======================>......] - ETA: 4s - loss: 2.3142 - acc: 0.0156  

In [ ]:
# JPEG compression with quality factor = 70
# JPEG compression with quality factor = 90
def jpeg_compression(img_path, q_factor):
    img = Image.open(img_path)
    img.save("../train_compress_/" + q_factor + folder + "/" + q_factor + "jpg_" + pic , "JPEG", quality=q_factor)

